In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

train1 = train.drop(['Descript', 'Resolution'], axis = 1)

train1 = train1.query('Y != 90')

In [ ]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

DOW_mapping = {'Monday':1, 'Tuesday':1, 'Wednesday':1, 'Thursday':1, 'Friday':1, 'Saturday':0, 'Sunday':0}
train1['DayOfWeek'] = train1['DayOfWeek'].map(DOW_mapping)

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

le = LabelEncoder()
train1['Category'] = le.fit_transform(train1['Category'])
train1
# DayOfWeek...平日休日mapping, PdDistrict...count

In [ ]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

DOW_mapping = {'Monday':1, 'Tuesday':1, 'Wednesday':1, 'Thursday':1, 'Friday':1, 'Saturday':0, 'Sunday':0}
test['DayOfWeek'] = test['DayOfWeek'].map(DOW_mapping)


count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)

In [ ]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [ ]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()

cb.fit(X_train, y_train)
y_pred = cb.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

In [ ]:
cb_submission = cb.predict(X_test)

In [ ]:
import csv
pred = cb.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("cat3.csv", index=False)